In [ ]:
# Libraries for plotting
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# load processed Nav data
import pandas as pd
nav = pd.read_hdf('../data/interim/nav_converted.h5', 'table')
paros = pd.read_hdf('../data/interim/paros_converted.h5', 'table')
sbe3 = pd.read_hdf('../data/interim/sbe3_converted.h5', 'table')

# nadv = pd.read_hdf('../data/interim/nortek_converted.h5', 'table')

# get start time
T0 = nav.head(1).epoch[0]

In [ ]:
#def area_select(df):
#    return df.loc[(df.xrot >= xmin) & (df.xrot <= xmax) & (df.yrot >= ymin) & (df.yrot <= ymax)]

def sel_by_depth(nav, depthmin=-5000, depthmax=-1000):
  return nav[(nav.depth < depthmax) & (nav.depth > depthmin)]

def sel_by_epoch(nav, T0, Tmin=-5000, Tmax=-1000):
  return nav[(nav.epoch < (Tmax+T0)) & (nav.epoch > (Tmin+T0))]

# Fit Paros to Nav

In [ ]:
def first_order_comp(x, m, c):
    return x*m + c

In [ ]:
nav['pressure'] = np.interp(nav.epoch.values, paros.epoch.values, paros.pressure.values)
nav_slice = sel_by_depth(nav, depthmin=-1550, depthmax=-1450).copy()
params = np.polyfit(nav_slice.pressure.values, nav_slice.depth.values, 1)
print('Fit Parameters: %0.10f %0.10f' % (params[0], params[1]))

# add paros predicted depth
nav['paros_depth'] = nav.pressure*params[0] + params[1]

In [ ]:
nav_slice_plot = sel_by_epoch(nav, T0, Tmin=2960, Tmax=5070)

fig, ax1 = plt.subplots(figsize=(12, 9), dpi=80)
hd2, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.paros_depth.values,'r', label='paros')
hd1, = ax1.plot(nav_slice_plot.epoch.values - T0, nav_slice_plot.depth.values, label='nav')

ax1.set(xlabel='time [s]', ylabel='depth [m]',
       title='Zoomed depth & temperature, bottom01, -1480m')
ax1.grid()

# ax2 = ax1.twinx()
# hd3, = ax2.plot(nav_slice.epoch.values - T0, sel_sbe3_t.temp_stinger.values, 'm', label='sbe3')
# ax2.set_ylabel('temp_stinger', color='m')
# ax2.tick_params('y', colors='m')

# fig.tight_layout()
# plt.show()
plt.legend(handles=[hd1, hd2])

# Function to Rotate ADV Estimate

In [ ]:
def e2q(E):
    q = np.array([0.0,0.0,0.0,0.0])
    hroll = E[0]/2.0
    hpitch = E[1]/2.0
    hyaw = E[2]/2.0
    sin_r2 = np.sin(hroll)
    sin_p2 = np.sin(hpitch)
    sin_y2 = np.sin(hyaw)
    cos_r2 = np.cos(hroll)
    cos_p2 = np.cos(hpitch)
    cos_y2 = np.cos(hyaw)
    q[0] = cos_r2 * cos_p2 * cos_y2 + sin_r2 * sin_p2 * sin_y2
    q[1] = sin_r2 * cos_p2 * cos_y2 - cos_r2 * sin_p2 * sin_y2
    q[2] = cos_r2 * sin_p2 * cos_y2 + sin_r2 * cos_p2 * sin_y2
    q[3] = cos_r2 * cos_p2 * sin_y2 - sin_r2 * sin_p2 * cos_y2
    q = q/np.linalg.norm(q)
    if q[0]<0:
        q = -q
    return q

#test
E = np.array([0.0,0.0,0.0])
if np.linalg.norm(np.array([1.0,0,0,0])-e2q(E)) > 1e-10:
    raise ValueError("e2q not working")

def q2R(q):
    # q = [scalar vector]
    w = q[0]
    x = q[1]
    y = q[2]
    z = q[3]

    nrm = np.linalg.norm(q)
    if (abs(nrm) < 0.9):
        disp('QuaternionLib::q2C -- not a unit quaternion');
        R = np.eye(3)
        return R
    nrm = 1./nrm
    w = w*nrm
    x = x*nrm
    y = y*nrm
    z = z*nrm
    x2 = x*x
    y2 = y*y
    z2 = z*z
    w2 = w*w
    xy = 2*x*y
    xz = 2*x*z
    yz = 2*y*z
    wx = 2*w*x
    wy = 2*w*y
    wz = 2*w*z
    R = np.zeros((3,3))
    R[0,0] = w2+x2-y2-z2 
    R[0,1] = xy-wz
    R[0,2] = xz+wy
    R[1,0] = xy+wz
    R[1,1] = w2-x2+y2-z2
    R[1,2] = yz-wx
    R[2,0] = xz-wy
    R[2,1] = yz+wx
    R[2,2] = w2-x2-y2+z2
    return R

# return quaternion equivalent to rotation matrix R
# q is wxyz
# from MIT libbot
def q2e(q):
    E = np.zeros(3)
    roll_a = 2 * (q[0]*q[1] + q[2]*q[3]);
    roll_b = 1 - 2*(q[1]*q[1] + q[2]*q[2]);
    E[0] = np.arctan2(roll_a, roll_b);
    pitch_sin = 2*(q[0]*q[2] - q[3]*q[1]);
    E[1] = np.arcsin(pitch_sin);
    yaw_a = 2*(q[0]*q[3] + q[1]*q[2]);
    yaw_b = 1 - 2*(q[2]*q[2] + q[3]*q[3]);
    E[2] = np.arctan2(yaw_a, yaw_b);
    return E

### Quick tests

In [ ]:
E = np.array([0.0,0.0,0.0])
q2R(e2q(E))

In [ ]:
q = np.array([0.0,1.0,0.0,0.0])
q2e(q)

# Get Paros and ADV Velocities

In [ ]:
# get delta time
dt = np.diff(nav.epoch)
nav["dt"] = np.append(dt, 0.0)
if len(nav.dt) != len(nav.epoch):
    raise ValueError("dt length does not match epochs")
# average samping time
print("mean dt=", np.mean((nav[nav.dt < 1.0]).dt))

In [ ]:
# Paros velocity
paros_vertvel = np.diff(nav.paros_depth)
nav['paros_vel'] = np.append(paros_vertvel, 0.0)

# Convert Sentry Nav Orientation data to Rotation Matrix

In [ ]:
# rotate all ADV velocities to local frame
for i in range(len(nav.epoch)):
    1+1